<a href="https://colab.research.google.com/github/asgutierrt/health_robot/blob/main/floorplans/to_geojson.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from skimage import io, measure, morphology
from skimage.io import imsave, imread
from matplotlib import pyplot as plt

# import/install necessary libraries
import pandas as pd
import json

data=pd.read_csv('unisabana_hospital_rooms_csv.csv')

# transform hospital layout into geo-referenced polygon
def generate_non_axis_aligned_rectangular_mesh(polygon,X,shape):
    # define grid shape and steps
    x_A, y_A, x_B, y_B, x_D, y_D = polygon
    min_x,min_y,max_x,max_y = shape
    num_rows = (max_y-min_y); num_cols=(max_x-min_x)
    x_step = (x_B - x_A) / num_cols; y_step = (y_D - y_A) / num_rows

    # transform and store each shape point
    mesh_points = []
    for x, y in X:
        x_ = x_A + (x-min_x) * x_step  + (y-min_y) * (x_D - x_A) / num_rows
        y_ = y_A + (y-min_y) * y_step  + (x-min_x) * (y_B - y_A) / num_cols
        mesh_points.append([x_, y_])
    return mesh_points

# load polygon information into geojson format
def process_type(attributes,shape):
    # save a list of each shape's edges [pixels]
    if shape=='rect':
        x=int(attributes['x']); y=int(attributes['y'])
        dx=int(attributes['width']); dy=int(attributes['height'])
        return [[x,y],[x+dx,y],[x+dx,y+dy],[x,y+dy]]
    else:
        x=attributes['all_points_x']; y=attributes['all_points_y']
        return [[int(x),int(y)] for x,y in zip(x,y)]

# load dictionary columns of information
dict_cols=['region_shape_attributes','region_attributes']
data[dict_cols]=data[dict_cols].applymap(lambda x: json.loads(x))

# process 'rect' and 'polygon' shapes
data['type']=data['region_shape_attributes'].apply(lambda x: x['name'])
for shape,df_shape in data.groupby('type', group_keys=False)['region_shape_attributes']:
    data.loc[df_shape.index,'pixls_coor']=df_shape.apply(process_type,args=[shape])

# get shape size (pixels)
maxpxl_x,maxpxl_y=pd.DataFrame(data['pixls_coor'].apply(max).values.tolist()).max()
minpxl_x,minpxl_y=pd.DataFrame(data['pixls_coor'].apply(min).values.tolist()).min()

# transform original map into hospital polygon coordinates (lat,lon)
x_D,y_D=-74.03056135394623,4.8563414479645814
x_B,y_B=-74.03052796587849,4.855884944267842
x_A,y_A=-74.03033456778033,4.856022342488572
polygon=[x_A, y_A, x_B, y_B, x_D, y_D]
shape=[minpxl_x,minpxl_y,maxpxl_x,maxpxl_y]
mesh=lambda X: generate_non_axis_aligned_rectangular_mesh(polygon,X,shape)
data[['coordinates']]=data[['pixls_coor']].applymap(lambda x: mesh(x))
data[['coordinates']]=data[['coordinates']].applymap(lambda x: x+[x[0]])

# save with geojson format
features=[]
for i,feature in data.iterrows():
    info={"type": "Feature",
            "properties":feature['region_attributes'],
            "geometry":{"coordinates":[feature['coordinates']], "type": "Polygon"},
            "id":i}
    features.append(info)

# download file (visualize on https://geojson.io/#new&map=2/0/20)
geojson={"type": "FeatureCollection", "features": features}
with open('unisabana_hospital.geojson', 'w') as convert_file:
    geojson_file=json.dumps(geojson)
    convert_file.write(geojson_file)


In [ ]:
from PIL import Image
img = Image.open('floorII_walls.png').convert("L")
bw = img.point(lambda x: 0 if x<220 else 255, '1')
data = np.asarray(bw)
bw

#!pip install rasterio
import rasterio.features

# Mask is a numpy array binary mask loaded however needed
maskShape = rasterio.features.shapes(data.astype('uint8'))
mypoly=[]
for vec in maskShape:
    mypoly.append(vec[0])

print(mypoly)

[{'type': 'Polygon', 'coordinates': [[(222.0, 58.0), (842.0, 58.0), (842.0, 234.0), (708.0, 234.0), (708.0, 279.0), (697.0, 279.0), (697.0, 362.0), (712.0, 362.0), (712.0, 664.0), (261.0, 664.0), (261.0, 338.0), (222.0, 338.0), (222.0, 337.0), (342.0, 337.0), (342.0, 231.0), (222.0, 231.0), (222.0, 58.0)], [(227.0, 62.0), (227.0, 104.0), (294.0, 104.0), (294.0, 106.0), (227.0, 106.0), (227.0, 148.0), (296.0, 148.0), (296.0, 151.0), (227.0, 151.0), (227.0, 188.0), (265.0, 188.0), (265.0, 165.0), (269.0, 165.0), (269.0, 188.0), (290.0, 188.0), (290.0, 174.0), (294.0, 174.0), (294.0, 191.0), (227.0, 191.0), (227.0, 227.0), (345.0, 227.0), (345.0, 337.0), (464.0, 337.0), (464.0, 385.0), (436.0, 385.0), (436.0, 382.0), (460.0, 382.0), (460.0, 368.0), (426.0, 368.0), (426.0, 364.0), (461.0, 364.0), (461.0, 340.0), (406.0, 340.0), (406.0, 383.0), (401.0, 383.0), (401.0, 340.0), (340.0, 340.0), (340.0, 383.0), (335.0, 383.0), (335.0, 340.0), (264.0, 340.0), (264.0, 447.0), (335.0, 447.0), (335

In [ ]:
# save with geojson format
import json

features=[]
for i,geometry in enumerate(mypoly[1:-1]):
    mypoly[i]['coordinates']=[mesh(geometry['coordinates'][0])]
    info={"type": "Feature",
            "properties":{"name":"floor","Id":"2"},
            "geometry":mypoly[i],
            "id":i}
    features.append(info)
# download file (visualize on https://geojson.io/#new&map=2/0/20)
geojson={"type": "FeatureCollection", "features": features}
with open('unisabana_hospital_floor.geojson', 'w') as convert_file:
    geojson_file=json.dumps(geojson)
    convert_file.write(geojson_file)

In [ ]:
i

2